## Question 13.2
***In this problem you, can simulate a simplified airport security system at a busy airport. Passengers arrive
according to a Poisson distribution with λ1 = 5 per minute (i.e., mean interarrival rate 1 = 0.2 minutes)
to the ID/boarding-pass check queue, where there are several servers who each have exponential
service time with mean rate 2 = 0.75 minutes. [Hint: model them as one block that has more than one
resource.] After that, the passengers are assigned to the shortest of the several personal-check queues,
where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1
minute).***

***Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the
system, and then vary the number of ID/boarding-pass checkers and personal-check queues to
determine how many are needed to keep average wait times below 15 minutes. [If you’re using SimPy,
or if you have access to a non-student version of Arena, you can use λ1 = 50 to simulate a busier airport.]***

To execute this simulation problem I am going to use the simpy package in python. I start by first setting up the project with basic package import as well as declaring some constant variables that were given to us in the outline above.

In [205]:
# import necessary libraries
import random
import simpy
import pandas as pd

# Create constant variables
check_time = 0.75   # Minutes it takes to service a passenger
min_scan_time = 0.5 # Min time it takes for a passenger to self scan
max_scan_time = 1.0 # Max time it takes for a passenger to self scan
sim_time = 60       # Simulation time in minutes

In the next 3 blocks of code I am going to set up the actual environment of the simulation. In the first code block below I first create the Airport environment object. This environment contains 3 functions, one to initialize the environment, one to check passengers through the boarding-pass checker process, and one to put passengers through the scan process. An airport has a limited number of boarding-pass checkers (num_checkers) to check passengers in parellel as well as a limited number of passenger scanners (num_scanners). Passengers have to queue at one boarding pass checker. When they got one, they can start the boarding pass check process and wait for it to finish which takes 0.75 minutes (check_time). They then move on to the queue at one scanner. When they get a scanner, they can start the scan process and wait for it to finish which takes between 0.5 and 1.0 minutes (scan_time).

In [211]:
class Airport(object):

    def __init__(self, env, num_checkers, check_time, 
                 num_scanners, min_scan_time, max_scan_time):
        self.env = env
        self.checker = simpy.Resource(env, num_checkers)
        self.check_time = check_time
        self.scanner = num_scanners
        self.min_scan_time = min_scan_time
        self.max_scan_time = max_scan_time

    def check(self, Passenger):
        """The check process. It takes a passenger and processes them"""
        yield self.env.timeout(check_time)
     
    def scan(self, Passenger):
        """The scan process. It takes a passenger and then scans them"""
        
        scan_time = random.uniform(min_scan_time, max_scan_time)
        yield self.env.timeout(scan_time)
        scan_times.append(scan_time)

This next code block continues the environment set up. The function below declares the Passenger behavior within an Airport environment. Each passenger has a name, arrives at the airport (Airport) and requests a boarding-pass checker. They then start the boarding-pass check process, waits for it to finish and then request a passenger scanner, where they wait until it finishes.

In [207]:
def Passenger(env, name, Airport):

    passengers.append(name)
    arrival_times.append(env.now)
    
    with Airport.checker.request() as request:
        yield request

        check_line_enter_times.append(env.now)
        yield env.process(Airport.check(name))

        scanner_line_enter_times.append(env.now)
        yield env.process(Airport.scan(name))
        
        scanner_line_exit_times.append(env.now)

The function below finishes the set up of the environment. The setup function creates an Airport, a number of initial passengers and keeps creating passengers approximately every 0.2 minutes (arrival_inter) for low volume airports, and 0.02 minutes for large airpots.

In [213]:
def setup(env, num_checkers, check_time, num_scanners, scan_time, arrival_inter):
    
    # Create the Airport
    airport = Airport(env, num_checkers, check_time, 
                      num_scanners, min_scan_time, max_scan_time)

    # Create 5 initial passengers
    for i in range(5):
        env.process(Passenger(env, 'Passenger %d' % i, airport))

    # Create more passengers while the simulation is running
    while True:
        yield env.timeout(arrival_inter)
        i += 1
        env.process(Passenger(env, 'Passenger %d' % i, airport))

After completing the set up of the simulation environment, the next step is to execute the simulation with a few iterable variables. I iterated through 2 lambda values of 5.0 and 50.0 which represent the passenger arrival rates at the airport (5 being low, 50 being high. I also iterated through the number of passenger checkers and scanners. I stored all of the results in a few different lists, calculated the total average wait time for each iteration, and then created a final wait_time_df below this next block.

In [214]:
# Setup and start the simulation
random.seed(23)

# Create list for average wait times to append into
avg_wait_times = []
num_checkers_scanners = []
arrival_rates = []

# Iterate through different passenger arrival rates to test airport business
for i in [5.0, 50.0]:
    
    # Set passenger arrival rate
    arrival_inter = 1.0 / i
    
    # Iterate through different combinations of checkers & scanners 
    # to optimize wait time below 15 minutes
    for j in range(1, 41):

        # Create lists for storing simulation results
        passengers = []
        arrival_times = []
        check_line_enter_times = []
        check_times = []
        scanner_line_enter_times = []
        scan_times = []
        scanner_line_exit_times = []

        num_checkers = j # number of boarding-pass checkers
        num_scanners = j # number of scanners

        # Create an environment and start the setup process
        env = simpy.Environment()
        env.process(setup(env, num_checkers, check_time, num_scanners, 
                          scan_time, arrival_inter))

        # Execute
        env.run(until=sim_time)

        # Store check wait time, scanner wait times and 
        # create df for total wait time
        check_line_wait_times = [abs(round((i - j) - 0.75,2)) 
                                 for i, j in zip(scanner_line_enter_times, 
                                                 arrival_times)]
                                                 
        scan_line_wait_times = [abs(round((i - j) - k, 2)) 
                                for i, j, k in zip(scanner_line_exit_times, 
                                                   scanner_line_enter_times, 
                                                   scan_times)]
                                                   
        total_wait_times = [i + j for i, j in zip(check_line_wait_times, 
                                                  scan_line_wait_times)]

        # Append total average wait time, number of checkers, and number of scanners to lists
        avg_wait_times.append(sum(total_wait_times) / len(total_wait_times))
        num_checkers_scanners.append(num_checkers)
        arrival_rates.append(arrival_inter)

The final dataframe containing the average wait time for each iteration of the number of checkers & scanners as well as the passenger arrival rates is below.

In [210]:
wait_time_df = pd.DataFrame(
    {'n_checkers_and_scanners': num_checkers_scanners,
     'avg_wait_time': avg_wait_times,
     'passenger_arrival_rate': arrival_rates
    })
    
wait_time_df.head(10)

,avg_wait_time,n_checkers_and_scanners,passenger_arrival_rate
0,26.006000,1,0.2
1,21.628077,2,0.2
2,17.769496,3,0.2
3,13.996879,4,0.2
4,10.421421,5,0.2
5,6.217712,6,0.2
6,1.936667,7,0.2
7,0.099394,8,0.2
8,0.079933,9,0.2
9,0.079360,10,0.2


Finally, in the next 2 code blocks I filtered the wait time dataframe from above to show the number of checkers & scanners required to keep the wait time below 15 minutes. The minimum number of checkers and scanners required for a low volume airport is 4, and the minimum required checkers and scanners for a high volume airport is 37. To improve upon this analysis, something that would be interesting to explore is the individual number of checkers versus scanners, instead of iterating through them at the same rate. We might find that either the checkers or the scanners might have a large impact on the wait time. This information would be most useful if we knew the max available of each of these categories, as well as the total cost of each to determine which value to minimize.

In [203]:
# Filter to low passenger arrival rate
low_arrival_rate = wait_time_df[wait_time_df['passenger_arrival_rate']==0.2]
low_arrival_rate = low_arrival_rate[low_arrival_rate['avg_wait_time'] <= 15.0]
low_arrival_rate.head(1)

,avg_wait_time,n_checkers_and_scanners,passenger_arrival_rate
3,13.996879,4,0.2


In [204]:
# Filter to high passenger arrival rate
high_arrival_rate = wait_time_df[wait_time_df['passenger_arrival_rate']==0.02]
high_arrival_rate = high_arrival_rate[high_arrival_rate['avg_wait_time'] <= 15.0]
high_arrival_rate.head(1)

,avg_wait_time,n_checkers_and_scanners,passenger_arrival_rate
76,14.815048,37,0.02
